In [115]:
import numpy as np
import pandas as pd
import warnings
import json
import requests
import random
import time
from elasticsearch import Elasticsearch
warnings.filterwarnings('ignore')

# 1. Exploračná analýza

## Tabuľka events

In [3]:
events = pd.read_csv('vi_data/events_train.csv')
events.sample(10)

,customer_id,timestamp,event_type,product_id,title,category_name,price
6544103,9d519c78-75c9-4ec0-9d61-f74767fe615c,2019-08-01 02:43:21.058 UTC,view_item,b345fd30-281d-4b32-8f88-76fc84bea664,Lee Cooper PU Biker Jacket Mens,Men>Men Clothing>Jackets & Coats,25.64
9502814,a9e2dea1-e183-471d-aa66-fdee5c2c3447,2019-08-12 10:45:41.02378 UTC,view_item,3a1cf659-e8a3-49c7-b1da-d0ea0fe82dd5,Lacoste Infant Unisex Marice Canvas Shoes,Children>Children's Footwear>Ballerinas,12.57
11560592,828538b3-b031-436e-a177-f2150514ac77,2019-08-20 16:58:56.308113 UTC,view_item,b124071d-de2f-471f-9078-4c4cc0450154,Lonsdale Leyton Leather Mens Trainers,Men>Men Footwear>Sneakers>Low sneakers,34.46
8213567,c74c7d0b-3807-44be-b917-349645729080,2019-08-07 10:42:30.455449 UTC,view_item,174b55d6-4747-4a77-9139-72588048f15f,Crafted Junior Boys 3 Pack Jog Set,Children>Boy's Clothing>Boy's sets,22.97
9540352,116464c9-fe13-465d-aede-3f641c645203,2019-08-12 13:06:32.629186 UTC,view_item,85bad8ed-d704-489b-bdae-c3061871a187,Lee Cooper Casual Woven Pants Mens,Men>Men Clothing>Tracksuit>Tracksuit bottoms,12.55
11794660,8fe7383f-83ea-4280-b2a5-9451d6397b18,2019-08-21 15:56:34.117964 UTC,view_item,de1b9182-defe-4c9f-8111-18fb8f1fa6c5,Lee Cooper Marl Backpack,Backpacks and Bags>Accessories >School Backpacks,13.02
3581409,9c7ad161-65eb-44bc-b888-d60f9ba57175,2019-07-18 05:10:08.540763 UTC,view_item,a6f866c8-a1eb-422a-a935-aa0646e54d13,Nike AOP W RunnerGrl91,Children>Girl's Clothing>Jackets,68.76
12297058,03de8819-d9aa-4933-ba5a-c88e3211e333,2019-08-23 14:08:48.424513 UTC,view_item,e27ddfad-5ffb-4826-ba3f-0e95d3d6b49b,Karrimor Ithaca Mens Walking Sandals,Men>Men Footwear>Sandals & Flip-Flops>Sandals,57.42
1819712,509a9bcb-c500-431c-affe-0d9feca00022,2019-07-09 19:02:58.958022 UTC,view_item,8d42785e-8bf8-4640-a731-787b8e4a6b7d,Miso Mixed Briefs Ladies,NaN,4.01
9588572,9ab2ab4f-57f8-4ff8-a982-f97857678fef,2019-08-12 16:23:07.870848 UTC,view_item,2295babf-d796-4986-b088-08ad33893880,Women's hight heels GEOX VIVYANNE HIGH D,Women>Women's Footwear>Court shoes,81.89


In [102]:
events.shape

(14614385, 7)

In [103]:
events.event_type.unique()

array(['view_item'], dtype=object)

## Tabuľka purchases

In [104]:
purchases = pd.read_csv('vi_data/purchases_train.csv')
purchases.sample(10) 

,customer_id,timestamp,event_type,product_id,title,category_name,price
188537,28689368-b934-4b01-9806-f51a24869fb1,2019-08-31 21:34:33.485056 UTC,purchase_item,2a5c9040-f73c-4531-bd75-51b580c2355b,Ombre Clothing Men's mid-season quilted jacket...,Men>Men Clothing>Jackets & Coats,26.00
35410,ec9bbec0-47f3-474b-b443-ddaebd61f2d2,2019-07-14 06:47:46.534368 UTC,purchase_item,11b5c302-a4e5-4f63-82f6-3b406685102b,Full Circle Poplin Long Shorts Ladies,NaN,NaN
128875,4edabc3c-8f11-4ddd-a212-e9580ecdcf23,2019-08-13 10:57:23.529451 UTC,purchase_item,0dde6d77-74d2-493f-9aad-b064febfabed,Firetrap Blackseal Long Line Bikini Top,Women>Women's Clothing>Swimmsuits & bikins,9.49
58594,899aeee2-c790-4da3-b9fa-5270b3a6b114,2019-07-22 14:04:58.742876 UTC,purchase_item,bd9a157e-7bc8-4f7c-a466-5385f96e48ed,Lee Cooper Fleece Shorts Mens,Men>Men Clothing>Shorts,11.38
143682,ccb2fc3d-ec18-4dfa-8c34-8fba74905ca6,2019-08-18 11:12:25.301356 UTC,purchase_item,03c9c3a8-5c84-4761-b1aa-52a62c4781f8,Nike Fav Leggings Junior Girls,Children>Girl's Clothing>Tracksuit & Leggings>...,18.21
155292,74ebb333-5948-4217-8d88-0ec3cbafa5d6,2019-08-21 22:22:43.62673 UTC,purchase_item,3f157c53-d2ae-4f6f-bc22-eef0335cc1fd,Crocheted headband DOKE HEADBAND,Women>Women's Accessories>Caps,11.77
21848,bc082c6b-74aa-40a9-8f46-6d122294093c,2019-07-09 08:06:26.711642 UTC,purchase_item,c5e73991-6c18-4176-9f12-925237f3e49a,Miso Rib Tank Top Ladies,NaN,NaN
23252,c77958ce-9071-438e-b004-2d4ba40f1ef6,2019-07-09 17:53:38.506568 UTC,purchase_item,a395c73b-2f47-4fcc-a749-fe0889d83489,Women's Jacket Marine,NaN,NaN
139859,a28a6b46-58d3-4e07-93f7-f7f53365ecfd,2019-08-16 20:01:08.510294 UTC,purchase_item,5c5df210-fec3-4ba1-a7e1-b0b734bec26c,Puma Epic V2 Flip JnC99,Women>Women's Footwear>Sliders & Flip-Flops>Fl...,16.12
33479,8dafc90f-9d6c-4bc3-9397-c8172dab6236,2019-07-13 10:57:08.081649 UTC,purchase_item,6bb56998-9fb1-443e-a8e8-62fe55199574,Lee Cooper Sling Dress Ladies,NaN,NaN


In [7]:
purchases.shape

(188712, 7)

In [105]:
purchases.event_type.unique()

array(['purchase_item'], dtype=object)

tabulky events a purchaches majú rovnaké stĺpce 

## Spojenie events and purchases

In [106]:
data = pd.concat([events, purchases], ignore_index=True)
data.sample(10)

,customer_id,timestamp,event_type,product_id,title,category_name,price
9786662,24a57718-3b21-46a1-bb85-795977466299,2019-08-13 12:12:46.572911 UTC,view_item,61e3a366-abfd-4f64-a6f9-7699a23ba760,Wallet WOOX Moneta Triviala,Women>Women's Accessories>Wallets,36.69
794201,dcd2e418-77eb-4d1c-88a1-22a761a6088d,2019-07-04 18:12:28.321765 UTC,view_item,9bb87057-23d6-47e3-ad97-46a6f98c5560,Trendyol Dusty rose glitter tulle detailed dress,NaN,52.70
6196100,5cd20652-736d-40d9-aaf1-080d57aa29ca,2019-07-30 16:06:43.631195 UTC,view_item,3b7c7c4b-12a2-4643-a838-5dcda0ad6838,Under Armour Perpetual Cap Mens,Men>Men Accessories>Baseball caps,34.35
2513664,302f5d55-6bcc-4198-8163-f13cdfa54940,2019-07-13 10:43:10.919824 UTC,view_item,7d896761-6132-4667-9781-b21f035a0c9f,Men's polo shirt B&amp;C Basic,NaN,8.43
4897142,cc740252-253f-431b-a08c-58272470582a,2019-07-24 17:03:07.627691 UTC,view_item,f8840c14-f913-4128-8d78-e8e6705f6f04,Garmin FR 245 Music Sn04,Men>Men Accessories>Watches,502.46
3243681,8ae9b194-06cd-443b-9f01-83b660c72859,2019-07-16 11:40:31.632604 UTC,view_item,1b0578ff-5b62-42d3-9bad-8f33ba341dd1,Puma Retaliate Mens Trainers,Men>Men Footwear>Sneakers>Sport sneakers,75.79
14791181,8b42aab2-1402-4605-a504-e643c001f7a6,2019-08-28 13:12:17.237471 UTC,purchase_item,1939227f-b47a-4faf-8a53-7234a7772866,Women's T-shirt Trendyol Flower Lace Detailed,Women>Women's Clothing>T-Shirts>Short Sleeve,18.23
11643565,2443879a-c908-42e2-a731-3dfc4cce2fae,2019-08-20 23:16:45.18704 UTC,view_item,01583de3-8a35-40fc-acbb-8fbe65e3b9de,SoulCal Deluxe Oxford Shirt,Men>Men Clothing>Shirts>Short sleeved shirts,13.12
13494897,74a909c9-b09d-43a3-ae52-c8f3660b3b23,2019-08-27 20:31:27.445261 UTC,view_item,e54a4c0c-38aa-47cb-bcc3-0291ce33425a,Character Gilet Set Girls,Children>Girl's Clothing>Girl's sets,38.15
8416741,f0a63d23-aec1-490a-9bae-25c6f669449b,2019-08-08 05:14:45.635668 UTC,view_item,860f347d-02cd-43aa-ad10-fdf8d0591eb4,Nike NSW Dress Tee Girls,Children>Girl's Clothing>Dresses & Skirts & Ov...,36.22


In [107]:
rows_num, cols_num = data.shape
print(rows_num)
print(cols_num)

14803097
7


In [108]:
del events
del purchases

## Atribúty

### customer_id

In [11]:
data.customer_id.describe()

count                                 14803097
unique                                 2211386
top       53f841c7-970b-4e0b-96bf-97a95ecfb734
freq                                      4804
Name: customer_id, dtype: object

In [12]:
len(data[data.customer_id.isna()])

0

In [13]:
rows_num/len(data.customer_id.unique())

6.694035776657716

In [14]:
data.groupby(['customer_id']).count()['timestamp'].median()

2.0

In [14]:
customer_rows = data['customer_id'].value_counts()
customer_rows

53f841c7-970b-4e0b-96bf-97a95ecfb734    4804
9eb76d09-8006-4262-b5eb-459556f338a9    3577
546a1146-64be-4549-a472-a8a4a8ca41b0    3189
a3fe2d55-b8fc-4aaa-8f7f-99f6498b191f    3060
68b2c520-5366-4143-a253-04fd96aeb251    2855
                                        ... 
28f3de37-5d91-451b-9d71-c016ff28765a       1
ddaea6ea-cf1e-4ef3-84d0-4d97f5019dab       1
ee2c4b69-2194-4785-a817-76a30ea8a54b       1
db9f09e8-ea9e-4444-986a-39c973a7dfe3       1
3fe6ea93-4d29-4d6f-990a-6a873f19c2ed       1
Name: customer_id, Length: 2211386, dtype: int64

In [28]:
for i, num in enumerate(customer_rows.value_counts()[:3]):
    print(i+1, num)

1 913643
2 329587
3 199175


In [29]:
(913643)/2211386

0.4131540129131685

In [20]:
(913643+329587)/2211386

0.5621949311427313

In [26]:
for i, num in enumerate(customer_rows.value_counts()[3:50:4]):
    print(4*i+4, num)

4 129721
8 45301
12 22832
16 13311
20 8728
24 6193
28 4481
32 3349
36 2677
40 2124
44 1784
48 1405


- 2,211,386 unikátnych zákazníkov
- žiadne NaN hodnoty
- v priemere vyše 6 záznamov na zákazníka
    - ale median je iba 2
    - 913,643 (41%) zákazníkov má len 1 záznam
    - 56% zákazníkov má 2 záznamy

### timestamp

In [16]:
data.timestamp.describe()

count                           14803097
unique                          14802889
top       2019-07-22 04:33:52.889148 UTC
freq                                   4
Name: timestamp, dtype: object

In [17]:
data.loc[data['timestamp'] == "2019-07-22 04:33:52.889148 UTC"]

,customer_id,timestamp,event_type,product_id,title,category_name,price
4361630,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,5bbe88b4-9650-4c1a-8f95-9a09c264b2c9,Hot Tuna Crew T Shirt Mens,Men>Men Clothing>T-Shirts & Polo shirts,12.84
4361631,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,248faf65-d1e2-4d7b-84c8-ce1c459ee086,Lonsdale Jersey Polo Shirt Mens,Men>Men Clothing>T-Shirts & Polo shirts>Polo S...,12.90
4361632,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,377bdb75-6f46-4c29-b559-9af0c6ba2ae8,Pierre Cardin Web Belt Shorts Mens,Men>Men Clothing>Shorts,17.12
4361633,8b57d536-f432-45a9-8b45-3bd54d68ba69,2019-07-22 04:33:52.889148 UTC,view_item,b2bcef2f-df2d-4c23-aee7-73ca112ef815,Lee Cooper Men's boxers 5-pack,Men>Men Clothing>Underwear>Boxer shorts & trunks,13.10


In [18]:
min(data.timestamp)

'2019-07-01 00:00:01.101977 UTC'

In [19]:
max(data.timestamp)

'2019-08-31 23:59:59.536067 UTC'

In [7]:
len(data[data.timestamp.isna()])

0

- dátumy z intervalu 2 mesiacov
- žiadne NaN hodnoty
- existujú záznamy typu `view_item`, kde je daný zákazník v danom čase a rôzne produkty. 

### event_type

In [21]:
data.event_type.unique()

array(['view_item', 'purchase_item'], dtype=object)

In [22]:
len(data[data.event_type.isna()])

0

In [23]:
len(data.loc[data['event_type'] == "view_item"])/rows_num

0.9872518568242848

In [24]:
len(data.loc[data['event_type'] == "purchase_item"])/rows_num

0.01274814317571519

- dva typy: `view_item` a `purchase_item`
- pomer je 98.7% ku 1.27%
- žiadne NaN hodnoty

### product_id

In [25]:
data.product_id.describe()

count                                 14803097
unique                                  123837
top       31ae211d-57a9-4bfa-b95f-40776af03c3f
freq                                     52736
Name: product_id, dtype: object

In [60]:
len(data[data.product_id.isna()])

0

In [61]:
rows_num/len(data.product_id.unique())

119.53694776197744

In [62]:
data.groupby(['product_id']).count()['timestamp'].median() 

17.0

In [32]:
product_rows = data['product_id'].value_counts()
product_rows

31ae211d-57a9-4bfa-b95f-40776af03c3f    52736
0f943312-7141-4606-abfa-81fd63a5498f    49141
a8f0292d-5fca-42b9-b0d3-b38e7efa416b    48343
e33cea18-ee3f-4e37-83a7-1d88c54d83b8    45421
608f32cd-210c-4af5-99d8-6182a9678cf5    39801
                                        ...  
6b0e71d0-cdca-4160-8df1-ae7b43ef967c        1
630d92d1-ed2b-4a75-a778-3dacba195eb5        1
0633f706-85d4-4ac6-b34d-7f653fa7139d        1
a3c46419-d8fb-4273-88ba-06dc92b42696        1
326e712f-9733-4761-b458-217f3f695833        1
Name: product_id, Length: 123837, dtype: int64

In [36]:
for i, num in enumerate(product_rows.value_counts()[:50:4]):
    print(4*i+1, num)

1 15982
5 3582
9 2365
13 1792
17 1376
21 1139
25 948
29 810
33 694
37 624
41 528
45 501
49 436


- 123,837 unikátnych produktov
- žiadne NaN hodnoty
- v priemere vyše 119 záznamov na produkt
    - maximum je 52,736
    - minimum je 1
    - median je 17
    - 15,982 produktov má len 1 záznam

### title

In [63]:
data.title.describe()

count                     14801284
unique                       71440
top       Lee Cooper Marl Backpack
freq                        168766
Name: title, dtype: object

In [72]:
data.loc[data['product_id'] == "31ae211d-57a9-4bfa-b95f-40776af03c3f"].title.unique()

array(['Batoh Lee Cooper Marl', 'Lee Cooper Marl Backpack'], dtype=object)

In [64]:
len(data[data.product_id.isna()])

0

- 71,440 unikátnych označení (našli sme 123,837 unikátnych produktov)
    - produkt môže mať viac rôznych označení
- žiadne NaN hodnoty

### price

In [85]:
data.price.describe().apply(lambda x: format(x, 'f'))

count    14758211.000000
mean           33.122065
std           121.532833
min             0.060000
25%            12.710000
50%            21.860000
75%            39.490000
max         32636.460000
Name: price, dtype: object

In [87]:
np.nanmedian(data.price)

21.86

In [73]:
len(data[data.price.isna()])

44886

In [89]:
data.loc[data['product_id'] == "31ae211d-57a9-4bfa-b95f-40776af03c3f"].price.unique()

array([12.9 , 12.73, 13.11, 13.32, 12.49, 12.71, 13.45, 12.65, 12.83,
       11.46, 11.8 , 11.44, 11.38, 12.1 , 11.24, 11.61, 11.99, 11.55,
        9.55,  9.56, 10.61, 10.63, 13.49,  7.32, 12.14, 12.27, 12.33,
       13.04, 12.92, 12.51, 12.11, 12.45, 12.35, 13.09, 16.84, 16.41,
       16.11, 16.03, 16.23, 15.95, 16.  , 15.58, 16.39, 16.97, 13.65,
       14.75, 13.98, 14.02, 14.05, 14.22, 14.38, 14.87, 14.12, 14.37,
       12.46, 12.37, 13.12, 12.04, 13.02, 12.39, 12.68, 12.55, 12.69,
         nan])

- priemer ceny je cca 33
- median ceny je cca 22
- 44,886 NaN hodnôt
- jeden produkt môže mať viac cien

### category_name

In [90]:
data.category_name.describe()

count                                   11367365
unique                                       713
top       Men>Men Footwear>Sneakers>Low sneakers
freq                                      462179
Name: category_name, dtype: object

In [103]:
for pid in data.product_id.sample(10):
    unique_categories = data.loc[data['product_id'] == pid].category_name.unique()
    print(len(unique_categories), unique_categories)

2 [nan 'Men>Men Clothing>T-Shirts & Polo shirts>Polo Shirts']
3 [nan "Women>Women's Clothing>Shirts"
 "Women>Women's Clothing>Shirts>Patterned shirts"]
2 [nan "Women>Women's Clothing>Jackets & Coats>Coats"]
2 [nan "Women>Women's Footwear>Slip-on"]
2 [nan "Women>Women's Clothing>Dresses>Wedding dress"]
2 [nan 'Men>Men Clothing>T-Shirts & Polo shirts>Polo Shirts']
2 [nan "Sports>Running>Running Shoes>Women's Running Shoes"]
2 [nan 'Accessories >Backpacks and Bags>Fashion backpacks']
3 ["Women>Women's Clothing>Pants & Jeans>Pants"
 "Women>Women's Clothing>Pants >Pants"
 "Women>Women's Clothing>Pants >Single-colored pants"]
2 [nan "Women>Women's Clothing>Jumpsuits"]


In [96]:
category_name_nans_num = len(data[data.category_name.isna()])
category_name_nans_num

3435732

In [95]:
category_name_nans_num/rows_num

0.23209548650529008

In [45]:
data["category_name"].str.split(">", expand = True).shape

(14803097, 7)

In [40]:
data_15K = data.sample(15000)

In [116]:
categories = data_15K["category_name"].str.split(">", expand = True)

Keďže `data` má skoro 15M záznamov, spracovanie trvá veľmi dlho. Vybrali sme len 15K.

In [123]:
categories.sample(5)

,0,1,2,3,4
2563389,NaN,NaN,NaN,NaN,NaN
7189282,Women,Women's Footwear,Balerinas,None,None
11478593,Men,Men Clothing,T-Shirts & Polo shirts,Short Sleeve,None
6003198,Women,Women's Clothing,Swimmsuits & bikins,One-piece,None
1495798,NaN,None,None,None,None


In [136]:
for i in range(5):
    print('col_{} unique values ='.format(i), len(categories[i].unique()))
    print('col_{} NaN values ='.format(i), len(categories[categories[i].isna()]))
    print()

col_0 unique values = 21
col_0 NaN values = 3455

col_1 unique values = 38
col_1 NaN values = 3551

col_2 unique values = 154
col_2 NaN values = 3669

col_3 unique values = 216
col_3 NaN values = 6845

col_4 unique values = 49
col_4 NaN values = 14513



In [126]:
categories[0].unique()

array([nan, 'Men', 'Women', 'Sports', "Women's Clothing", 'Children',
       'Accessories ', 'Backpacks and Bags', 'Men Clothing',
       "Boy's Clothing", 'Sale - Men', 'Sale - Women', 'Dresses',
       'NEW IN', 'Unlisted', "Children's Footwear", 'strom kategorii',
       'T-Shirts', 'NY FASHION WEEK', 'Sale - Children',
       'Delivery Immediately'], dtype=object)

- **iba** 713 unikátnych kategórií (našli sme 123,837 unikátnych produktov)
- jeden produkt môže mať rôzne `category_name` hodnoty, vrátane NaN
- 23.2% záznamov je NaN

- po rozparsovaní stromu `category_name` vzniklo 7 stĺpcov (t.j. výška stromu)
- smerom od koreňa rastie počet NaN hodnôt a unikátnych hodnôt
- vyskytujú sa tématicky rovnaké len inak napísané kategórie


### Diskusia
- Produkty zákazníka sa môžu opakovať, a preto je vhodné odporúčať už pozerané produkty
- Potreba zjednotiť nejednotné atribúty rovnakého produktu:
    - `title`
    - `category_name`
    - možno `cena`

---

# 2. Zjednotenie a spracovanie dát

In [7]:
categories = data["category_name"].str.split(">", expand = True)
del data["category_name"]
category_names = {}
for i in range(7):
    category_names[i] = "category_{}".format(i) 
categories = categories.rename(columns=category_names)
data = pd.concat([data, categories], axis=1, sort=False)
del categories

Vyparsovanie `category`

In [88]:
# Uloženie medzi-výsledku
# data.to_csv('vi_data/data_train.csv', encoding='utf-8')
# data = pd.read_csv('vi_data/data_train.csv', index_col=0)

In [219]:
data.sample(3)

,customer_id,timestamp,event_type,product_id,title,price,category_0,category_1,category_2,category_3,category_4,category_5,category_6
13029308,a307441c-2131-4cfe-a3f5-ceba85324cdb,2019-08-26 11:38:54.520998 UTC,view_item,7e40885a-c1e4-48e3-80e3-71951fc25a44,Children's Ski jacket LOAP FALDA,39.52,Children,Boy's Clothing,Jackets,Winter jackets,NaN,NaN,NaN
9634041,0e42ec0e-8d82-4d4a-b6ef-17ef06e3de1a,2019-08-12 19:25:21.494362 UTC,view_item,43bee0d7-2d3f-42ac-bef2-99de2b660be2,Top Secret LADY'S DRESS,15.34,Women,Women's Clothing,Dresses,Mini dress,NaN,NaN,NaN
14697110,7c04bdf3-0ea1-4830-9047-0350d7071ef2,2019-07-31 09:10:17.624419 UTC,purchase_item,ca3ccb9a-933f-45fb-a10a-dfde4eb046ee,Everlast Heavy Bomber Jacket Mens,35.00,Men Clothing,Men,Jackets & Coats,Bomber jackets,NaN,NaN,NaN


## Vytvorenie elastic indexu a naplnenie produktami

**PUT http://localhost:9200/vi_products**
```json
{'settings': {'number_of_shards': 1, 'number_of_replicas': 0},
 'mappings': {'properties': {'product_id': {'type': 'keyword'},
   'title': {'type': 'text', 'analyzer': 'english'},
   'price': {'type': 'double'},
   'events_number': {'type': 'integer'},
   'category_0': {'type': 'keyword'},
   'category_1': {'type': 'keyword'},
   'category_2': {'type': 'keyword'},
   'category_3': {'type': 'keyword'},
   'category_4': {'type': 'keyword'},
   'category_5': {'type': 'keyword'},
   'category_6': {'type': 'keyword'}}}}
```

In [194]:
product_ids = list(data.product_id.unique())

In [195]:
len(product_ids)

123837

In [192]:
def fill_index():
    es = Elasticsearch([{'host': '127.0.0.1', 'port': 9200}])
    doc = {}
    for pid in product_ids:
        rows = data[data.product_id == pid]
        doc['product_id'] = pid
        try:
            doc['title'] = list(rows.title.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['price'] = np.mean(rows.price.dropna().unique())
        except IndexError:
            pass
        try:
            doc['events_number'] = len(rows)
        except IndexError:
            pass
        try:
            doc['category_0'] = list(rows.category_0.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_1'] = list(rows.category_1.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_2'] = list(rows.category_2.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_3'] = list(rows.category_3.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_4'] = list(rows.category_4.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_5'] = list(rows.category_5.dropna().unique())[0]
        except IndexError:
            pass
        try:
            doc['category_6'] = list(rows.category_6.dropna().unique())[0]
        except IndexError:
            pass
        if np.isnan(doc['price']):
            del doc['price']
        es.index(index='vi_products', body=doc)

In [218]:
# fill_index()

---

# 3. Obsahový odporúčač

Navrhli sme obsahový odporúčač, ktorý používa nástroj Elasticsearch a dopyt [More Like This](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-mlt-query.html).

In [101]:
def ES_recommender_should(product_ids, fields=["title", "category_0", "category_1", "category_2", "category_3"], boosting_recent=False, count=10):
    """
    Using elasticsearch more_like_this query with should clause
    """
    url = 'http://localhost:9200/vi_products/_search'
    def transform_id(pid):
        data = {'query': {'bool': {'filter': {'term': {'product_id': pid}}}}}
        r = requests.post(url, json=data)
        try:
            return json.loads(r.text)['hits']['hits'][0]['_id']
        except IndexError:
            return None
    
    ids = [transform_id(pid) for pid in product_ids if pid]
    should = []
    for i, _id in enumerate(ids):
        boost = i if boosting_recent else 1
        if _id:
            mlt = {
              "more_like_this": {
                "fields": fields,
                "like": [
                  {
                    "_index": "vi_products",
                    "_id": _id
                  }
                ],
                "min_term_freq": 1,
                "min_doc_freq": 1,
                "max_query_terms": 100,
                "include": True,
                "boost": boost
              }
            }
            should.append(mlt)
            
    data = {
              "from": 0,
              "size": 10,
              "query": {
                "bool": {
                  "should": should
                }
              }
            }    
#     print(json.dumps(data))
    r = requests.post(url, json=data)
    try: 
        return [hit['_source']['product_id'] for hit in json.loads(r.text)['hits']['hits']]
    except KeyError as e:
        print('KeyError:'+e, r.text) 

Odporúča produkty, ktoré sú v súčte najpodobnejšie všetkým z trénovacím produtov

In [88]:
def ES_recommender_dis_max(product_ids, fields=["title", "category_0", "category_1", "category_2", "category_3"], boosting_recent=False, count=10):
    """
    Using elasticsearch more_like_this query with dis_max clause
    """
    url = 'http://localhost:9200/vi_products/_search'
    def transform_id(pid):
        data = {'query': {'bool': {'filter': {'term': {'product_id': pid}}}}}
        r = requests.post(url, json=data)
        try:
            return json.loads(r.text)['hits']['hits'][0]['_id']
        except IndexError:
            return None
    
    ids = [transform_id(pid) for pid in product_ids if pid]
    queries = []
    for i, _id in enumerate(ids):
        boost = i if boosting_recent else 1
        if _id:
            mlt = {
              "more_like_this": {
                "fields": fields,
                "like": [
                  {
                    "_index": "vi_products",
                    "_id": _id
                  }
                ],
                "min_term_freq": 1,
                "min_doc_freq": 1,
                "max_query_terms": 100,
                "include": True,
                "boost": boost
              }
            }
            queries.append(mlt)
            
    data = {
                "from":0,
                "size":10,
                "query": { 
                    "dis_max": {
                       "queries": queries
                    }
                }
            }   
#     print(json.dumps(data))
    r = requests.post(url, json=data)
    try: 
        return [hit['_source']['product_id'] for hit in json.loads(r.text)['hits']['hits']]
    except KeyError as e:
        print('KeyError:'+e, r.text) 

Odporúča produkty, ktoré sú najpodobnejšie niektorému z trénovacích produtov

# 4. Evaluácia

## Metriky

In [57]:
def precision_at_k(r, k=10):
    """Score is precision @ k
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Precision @ k
    Raises:
        ValueError: len(r) must be > 0
    """
    k = min(k, len(r))
    assert k > 0
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def dcg_at_k(r, k=10):
    """Score is discounted cumulative gain (dcg)
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
    Returns:
        Discounted cumulative gain
    Raises:
        ValueError: len(r) must be > 0
    """
    k = min(k, len(r))
    assert k > 0
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def ndcg_at_k(r, k=10):
    """Score is normalized discounted cumulative gain (ndcg)
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
    Returns:
        Normalized discounted cumulative gain
    Raises:
        ValueError: len(r) must be > 0
    """
    k = min(k, len(r))
    assert k > 0
    idcg = dcg_at_k(sorted(r, reverse=True), k)
    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

Upravené z [Ranking Metrics](https://gist.github.com/bwhite/3726239)

In [64]:
def evaluate(customer_id, recomender_function, metric_function):
    """
    Split customer's product, call recomender_function, compure relevance and metric_function
    """
    def split_products(products):
        middle = (len(products)+1)//2
        return [products[:middle], products[middle:]]
    customer_products = events[events.customer_id == customer_id].sort_values(by ='timestamp')
    if len(customer_products) == 1:
        raise Exception(customer_id + ' has just one product')
    product_ids = list(customer_products.product_id)
    train, test = split_products(product_ids)
    recomended = recomender_function(train, 
                                     fields=["title", "category_0", "category_1", "category_2", "category_3", "category_4", "category_5", "category_6"], 
                                     boosting_recent=True, 
                                     count=10)
    r = list()
    k = min(len(test), len(recomended))
    test = test[:k]
    recomended = recomended[:k]
    for i in range(k):
        r.append(1 if recomended[i] in test else 0)
    rank = metric_function(r)
    return rank #[train, test, recomended, r, rank]

Na vygenerovenie odporúčania bola použitá tabuľka `events`

## Výsledky

In [9]:
customer_ids = sorted(list(events.customer_id.unique()), key=lambda k: random.random())

In [58]:
len(customer_ids)

2210171

In [59]:
def results(recomender_function, metric_function, NUMBER_TESTED=10):
    start = time.time()
    p = []
    for cid in customer_ids:
        try:
            p.append(evaluate(cid, recomender_function, metric_function))
        except:
            pass
        if len(p) >= NUMBER_TESTED:
            break
    print('seconds:', time.time()-start)
    print('type:', recomender_function.__name__)
    print('number of tested customers:', len(p))
    print('average', metric_function.__name__,':', np.mean(p))

In [60]:
results(ES_recommender_should, ndcg_at_k, 300) #ES_recommender_should, title + 4 categoties

seconds: 484.64383912086487
type: ES_recommender_should
number of tested customers: 300
average ndcg_at_k : 0.2508594652179127


In [61]:
results(ES_recommender_dis_max, ndcg_at_k, 300) #ES_recommender_dis_max, title + 4 categoties

seconds: 691.1087830066681
type: ES_recommender_dis_max
number of tested customers: 300
average ndcg_at_k : 0.327475554946516


In [63]:
results(ES_recommender_dis_max, ndcg_at_k, 300) #ES_recommender_dis_max, title + all categoties

seconds: 622.2124481201172
type: ES_recommender_dis_max
number of tested customers: 300
average ndcg_at_k : 0.36574870105544943


In [70]:
results(ES_recommender_dis_max, ndcg_at_k, 300) #ES_recommender_dis_max, title + all categoties + boosting_recent

seconds: 710.6878938674927
type: ES_recommender_dis_max
number of tested customers: 300
average ndcg_at_k : 0.403434364394986


| **method**                                                       | **ndcg_at_10, average(300)** |
|------------------------------------------------------------------|------------------------------|
| ES_recommender_should, title + 4 categoties                      | 0.25085                      |
| ES_recommender_dis_max, title + 4 categoties                     | 0.32747                      |
| ES_recommender_dis_max, title + all categoties                   | 0.36574                      |
| ES_recommender_dis_max, title + all categoties + boosting_recent | 0.40343                      |

# 5. vi-challenge kaggle

In [110]:
vi_challenge_uID = pd.read_csv('vi_challenge_uID.csv', header=None)
len(vi_challenge_uID)

1000

In [114]:
start = time.time()
output = list()
most_popular_products = list(data.product_id.value_counts().index[:10])
for i, uid in enumerate(vi_challenge_uID[0]):
    customer_products = data[data.customer_id == uid].sort_values(by ='timestamp')
    product_ids = list(customer_products.product_id)
    if product_ids:
        recomended = ES_recommender_dis_max(product_ids,
                                            fields=["title", "category_0", "category_1", "category_2", "category_3", "category_4", "category_5", "category_6"], 
                                            boosting_recent=False, 
                                            count=10)
#         print(i, ": ", len(recomended))
        for r in recomended:
            output.append([uid, r])
    else:
#         print(i, "MPP")
        for r in most_popular_products:
            output.append([uid, r])
# print('seconds:', time.time()-start)

In [112]:
submission = pd.DataFrame(output, columns=['customer_id', 'product_id'])
submission.to_csv('harnusekSubmission.csv', index=False)
len(submission)

10000

##### [Naše najlepšie skóre v súťaži:  0.10909](https://www.kaggle.com/harnusek)
- použitie `ES_recommender_dis_max, title + all categoties`

# 6. Zhodnotenie

1. Počas exploračnej analýzy datasetu sme vypočítali štatistické údaje pre každý stĺpec a identifikovali sme potrebu ďalšieho predspracovania

2. Následne sme vyparsovali stĺpec `category` na 7 stĺpcov/kategórií, zjednotli sme atribúty rovnakých produktov a vložili všetky produkty do Elasticsearch indexu

3. Navrhli sme obsahový odporúčač, ktorý používa Elasticsearch dopyt [More Like This](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-mlt-query.html)
    - vytvorili sme 2 varianty odporúčača:
        - `ES_recommender_should`: odporúča produkty, ktoré sú v súčte najpodobnejšie všetkým z trénovacím produtov
        - `ES_recommender_dis_max`: odporúča produkty, ktoré sú najpodobnejšie niektorému z trénovacích produtov
    - ich vstupom sú mená stĺpce na základe ktorých sa určuje podobnosť 
    - je možné použiť "boostovanie" podobnosti produktov od najnovšie pozeraných po najstaršie (`boosting_recent`)
    
4. Počas evaluácie sme porovnávali oba viarianty a viaceré nastavenia parametrov s použitím metrík `precision_at_k` a `ndcg_at_k`
    - Podľa našich testov je najúspešnejší `ES_recommender_dis_max` s použitím `title`, všetkými kategóriami a `boosting_recent`
5. V kaggle súťaži sme dosiahli najlepšie skóre: [0.10909](https://www.kaggle.com/harnusek)
    - použitie `ES_recommender_dis_max` s použitím `title` a všetkými kategóriami

## Výhody riešenia
- vygenerovanie odporúčaní je pomerne rýchle (1000 zákazníkov trvá cca 20 minút)
- naše riešenie umožnuje jednoduché pridanie ďalších produktov, bez nutnosti pretrénovania
- možnosť jednoduchého experimentovania s parametrami

## Nevýhody riešenia
- zjednotenie dát a naplnenie indexu bolo veľmi pomalé (>24hodín)
- použitie len prístupu založenom na obsahu (v budúcnosti: možnosť použitia popularity produktov `events_number` v indexe)
- nepoužitie cien produktov